# Multi-Agent Orchestration

In this notebook, you'll create a team of agents that can create detailed research reports.

## 1. Prerequisites

- Python 3.12 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U -r requirements.txt

In [ ]:
import logging

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Enables Strands warnings log level
logging.getLogger("strands").setLevel(logging.WARNING)

In [ ]:
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

## 2. Experimentation

In [ ]:
from strands import Agent
from gather_evidence import gather_evidence_tool


agent = Agent(model=MODEL_ID, tools=[gather_evidence_tool])

response = agent.tool.gather_evidence_tool(pmcid="PMC9438179", question="How safe and effective are GLP-1 drugs for long term use?")


In [ ]:
formatted_response = {
    "toolUseId": response.get('toolUseId'),
    "status": response.get('status'),
    "pmcid": response['content'][1]['json']["pmcid"],
    "citation": response['content'][1]['json']["citation"],
    "question": response['content'][1]['json']["question"],
    "answer": response['content'][0]['text'],
    "evidence": [evidence['context'] for evidence in response['content'][1]['json']['evidence']]
}
formatted_response

### 2.1. DynamoDB stuff

Create table

In [ ]:
import boto3

dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "research-evidence"
try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table '{table_name}' already exists")
except:
    # Create table if it doesn't exist
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[{"AttributeName": "toolUseId", "KeyType": "HASH"}],
        AttributeDefinitions=[
            {"AttributeName": "toolUseId", "AttributeType": "S"},
            {"AttributeName": "pmcid", "AttributeType": "S"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "pmcid-index",
                "KeySchema": [{"AttributeName": "pmcid", "KeyType": "HASH"}],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST",
    )

    table.wait_until_exists()
    # print(f"Table '{table_name}' created")

Insert record

In [ ]:
# Example record

example = {
    "toolUseId": "tooluse_gather_evidence_tool_242995659",
    "status": "success",
    "pmcid": "PMC9438179",
    "citation": 'Nauck, Michael A., and David A. D\'Alessio. "Tirzepatide, a dual GIP/GLP-1 receptor co-agonist for the treatment of type 2 diabetes with unmatched effectiveness regrading glycaemic control and body weight reduction." *Cardiovascular Diabetology*, vol. 21, no. 169, 1 Sept. 2022, doi:10.1186/s12933-022-01604-7.',
    "question": "How safe and effective are GLP-1 drugs for long term use?",
    "answer": "Based on the available evidence, GLP-1 receptor agonists demonstrate favorable safety and efficacy profiles for long-term use. Specifically, tirzepatide, a dual GLP-1/GIP receptor co-agonist, shows sustained glycemic control and weight loss over 52-week treatment periods (Nauck2022 chunk 2). The safety profile appears acceptable, with adverse event rates and cardiovascular outcomes comparable to existing GLP-1 receptor agonist comparators (Nauck2022 chunk 2).\n\nThe long-term effectiveness is characterized by maintained glycemic control and continued weight reduction benefits throughout the 52-week study duration (Nauck2022 chunk 2). From a safety perspective, the cardiovascular outcomes data suggest no increased risk compared to established GLP-1 receptor agonist therapies (Nauck2022 chunk 2).\n\nHowever, the provided context is limited to data from tirzepatide studies over a 52-week period. While this dual agonist represents an advancement in the GLP-1 therapeutic class with unmatched effectiveness regarding glycemic control and body weight reduction, comprehensive long-term safety data beyond one year and broader evidence across the entire class of GLP-1 receptor agonists would be needed for a more complete assessment of long-term safety and efficacy (Nauck2022 chunk 2).",
    "evidence": [
        "In 26‑ to 40‑week trials, GLP‑1 agonists dulaglutide (1.5\u202fmg) and semaglutide (1\u202fmg) lowered HbA1c by ~1.1–1.9% and reduced weight by 2.7–7.8\u202fkg. Gastro‑intestinal events occurred in 30‑40% of patients, with 4‑11% discontinuing, indicating good long‑term efficacy and acceptable safety.",
        "Phase‑2/3 trials of tirzepatide, a GLP‑1/GIP co‑agonist, demonstrate sustained glycaemic control and weight loss over 52‑week periods, with safety data—including adverse‑event rates and cardiovascular outcomes—comparable to GLP‑1RA comparators, indicating effective and acceptable long‑term use.",
        "Tirzepatide, a dual GIP/GLP‑1 agonist, provides robust HbA1c reductions (up to\u202f2.4%), significant weight loss, and a favorable cardiovascular profile (MACE hazard ratios below 1.3, no excess risk). Its 5‑day half‑life supports weekly dosing, and pharmacokinetics are unchanged in renal or hepatic impairment, indicating good long‑term safety and efficacy.",
        "Long‑term GLP‑1 receptor agonists (e.g., semaglutide) consistently lower HbA1c and body weight, improve insulin sensitivity, and reduce appetite. Dual GIP/GLP‑1 agonist tirzepatide shows even greater, sustained reductions in glucose and weight, with no major safety signals reported, though data on GIP‑related effects remain limited.",
        "Clinical data show GLP‑1‑based therapies provide durable glycaemic control, significant weight loss, and improved cardiovascular risk biomarkers over months to years, with a safety profile dominated by transient nausea that can be mitigated (e.g., by GIP co‑agonism).",
    ],
}

import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(table_name)

response = table.put_item(
    Item=example
)


Get record

In [ ]:
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(table_name)

response = table.get_item(
    Key={
        'toolUseId': 'tooluse_gather_evidence_tool_242995659'
    }
)
response.get("Item")

Query by secondary index

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource("dynamodb")
table = dynamodb.Table(table_name)

response = table.query(
    IndexName="pmcid-index", KeyConditionExpression=Key("pmcid").eq("PMC9438179")
)
response.get("Items")